In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from pymongo import MongoClient
from tqdm import tqdm
import time
from datetime import datetime

In [9]:
# MongoDB 클라이언트 및 컬렉션 설정
client = MongoClient(host="1.234.51.110", port=38019, username='clawling', password='goodtime**95')
db = client['BOF2024_crawling']
collection = db['naver_tv_crawling']

In [10]:
# 웹드라이버 초기화 및 네이버tv 검색 페이지 열기
driver = webdriver.Chrome()
driver.get("https://tv.naver.com/")
driver.maximize_window()

# 3초간 대기
time.sleep(3)

In [11]:
# 검색어 입력 요소 찾기
search_input = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "__gnb__searchInput")))

# 검색어 입력
search_input.send_keys("Busan One Asia Festival")

# 3초간 대기 후 엔터 키 입력
time.sleep(3)
search_input.send_keys(Keys.RETURN) 

In [12]:
# 조회수 형식을 변환 (ex) 1.3만 → 13000 형식으로 변환)
def convert_view_count(view_count_text):
    view_count_text = view_count_text.replace(',', '')  # 쉼표 제거
    if '만' in view_count_text:
        count = int(float(view_count_text.replace('만', '')) * 10000)
    else:
        count = int(view_count_text)
    return count

In [13]:
# 3초간 대기
time.sleep(3)

# 맨 아래까지 스크롤
SCROLL_PAUSE_TIME = 1.5
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.documentElement.scrollHeight;")
    if new_height == last_height:
        break
    last_height = new_height

In [14]:
try:
    # 영상 정보 가져오기
    video_datas = driver.find_elements(By.CSS_SELECTOR, '.ClipHorizontalCardV2_list_item__dtxfs')

    for data in tqdm(video_datas, desc="동영상 수집 진행 중"):
        # 영상 제목 요소 찾기
        video_title_elem = data.find_element(By.CSS_SELECTOR, '.ClipHorizontalCardV2_title__4gz8z')
        video_title = video_title_elem.text.strip()

        # 영상 URL 요소 찾기
        video_url_elem = data.find_element(By.CSS_SELECTOR, '.ClipHorizontalCardV2_link_title__v1EH7')
        video_url = video_url_elem.get_attribute('href')
        
        # 채널명 요소 찾기
        channel_name_elem = data.find_element(By.CSS_SELECTOR, '.ClipHorizontalCardV2_channel_name__XaLUw')
        channel_name = channel_name_elem.text.strip()

        # 조회수 요소 찾기
        try:
            view_count_elem = data.find_element(By.CSS_SELECTOR, '.ClipHorizontalCardV2_text__4iw73')
            view_count_text = view_count_elem.text.strip()
            # 숫자가 아닌 경우 해당 데이터를 건너뜀 (ex) "19세 이상" 등으로 조회수가 표기된 경우 존재)
            if not view_count_text.replace('만', '').replace(',', '').replace('.', '').isdigit():
                continue
            view_count = convert_view_count(view_count_text)
        except NoSuchElementException:
            view_count = 0  # 조회수 요소가 없는 경우 0을 할당

        # 설명 요소 찾기
        video_description_elem = data.find_element(By.CSS_SELECTOR, '.ClipHorizontalCardV2_description__RBDmS')
        video_description = video_description_elem.text.strip()

        # 영상 길이 요소 찾기
        video_length_elem = data.find_element(By.CSS_SELECTOR, '.ClipHorizontalCardV2_playtime__smy9H')
        video_length = video_length_elem.text.strip()

        # MongoDB에 데이터 적재
        data = {
            "제목": video_title,
            "URL": video_url,
            "채널명": channel_name,
            "조회수": view_count,
            "설명": video_description,
            "영상 길이": video_length,
            "검색키워드": "Busan One Asia Festival"  # 검색 키워드를 지정 
        }
        collection.insert_one(data)

finally:
    # 작업 완료 후 드라이버 닫기
    driver.quit()
    client.close()

동영상 수집 진행 중: 100%|██████████| 139/139 [00:11<00:00, 12.46it/s]
